1.1

In [159]:
import pandas
features = pandas.read_csv('./features.csv', index_col='match_id')



cols_to_del=['duration','radiant_win','tower_status_radiant','tower_status_dire', 'barracks_status_radiant','barracks_status_dire']

X=features.drop(cols_to_del, axis=1, inplace=False)
target=features['radiant_win']

 1.2 пропуски в данных встречаются в столбцах ниже. Столбцы с префиксом "first_" обозначают события которые должны произойти в первые 5 минут игры. 
Например поле first_blood_time означает что первая кровь была пролита спустя больше чем 5 минут
Другие поля означают события которые могли не наступить по ходу игры. Например dire_bottle_time - время 1й покупки предммета  
bottle командой dire. Пропуск означает что в ходе игры команда dire предмет bottle не купила

In [160]:
X.columns[X.count()!=X.shape[0]]


Index([u'first_blood_time', u'first_blood_team', u'first_blood_player1',
       u'first_blood_player2', u'radiant_bottle_time', u'radiant_courier_time',
       u'radiant_flying_courier_time', u'radiant_first_ward_time',
       u'dire_bottle_time', u'dire_courier_time', u'dire_flying_courier_time',
       u'dire_first_ward_time'],
      dtype='object')

1.3 

In [161]:
X=X.fillna(0)

1.4 Целевая переменная - столбец radiant_win

In [162]:
target=features['radiant_win']

1.5

In [37]:
import time
import datetime
import sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

In [163]:
data=X
data['y']=target


In [165]:

n_all=[250]#[10,100,200,300]
for n_val in n_all:
        i=0
        n=n_val
        my_max_depth=3
        my_n_splits=5
        my_shuffle=True
        my_random_state=17
        kf = KFold(n_splits=my_n_splits
                  ,shuffle=my_shuffle
                  ,random_state=my_random_state)
        auc_all=0
        start_time0 = datetime.datetime.now()
        for train, test in kf.split(data):
            start_time = datetime.datetime.now()
            train_data = data.iloc[train]
            test_data = data.iloc[test]
            X_train, target_train=train_data[train_data.columns[train_data.columns!='y']], train_data['y']    
            X_test, target_test=test_data[test_data.columns[test_data.columns!='y']], test_data['y']
            model=GradientBoostingClassifier(n_estimators=n
                                        ,max_depth=my_max_depth
                                        ,random_state=my_random_state)
            model.fit(X_train,target_train)
            target_predicted=model.predict(X_test)
            my_roc_auc_score=roc_auc_score(target_test, target_predicted)
            my_str='Time elapsed: '+ str(datetime.datetime.now() - start_time)+str(', n=')+str(n)+str(', roc_auc=')+str(my_roc_auc_score)+str(', i=')+str(i)
            i += 1
            auc_all += my_roc_auc_score
            print my_str

        my_str_result= 'Time elapsed: '+ str(datetime.datetime.now() - start_time0)+str(', n=')+str(n)+str(', roc_auc_avg=')+str(auc_all*1.0/max(i,1))
        print my_str_result

Time elapsed: 0:01:44.396111, n=250, roc_auc=0.65446686537, i=0
Time elapsed: 0:01:50.725687, n=250, roc_auc=0.646660490922, i=1
Time elapsed: 0:01:52.642778, n=250, roc_auc=0.657372297768, i=2
Time elapsed: 0:01:45.753377, n=250, roc_auc=0.648240772688, i=3
Time elapsed: 0:01:50.571482, n=250, roc_auc=0.654289639043, i=4
Time elapsed: 0:09:04.100023, n=250, roc_auc_avg=0.652206013158


2.1

In [82]:
from sklearn.linear_model import LogisticRegression

In [148]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X))
X_scaled.columns=X.columns

In [154]:
data=X_scaled
data['y']=target
data=data.fillna(0)

In [156]:
C_vals=[10**i for i in range(-5,5)]

for c in C_vals:
        i=0
        n=c
        #my_max_depth=3
        my_n_splits=5
        my_shuffle=True
        my_random_state=17
        kf = KFold(n_splits=my_n_splits
                  ,shuffle=my_shuffle
                  ,random_state=my_random_state)
        auc_all=0
        start_time0 = datetime.datetime.now()
        for train, test in kf.split(data):
            start_time = datetime.datetime.now()
            train_data = data.iloc[train]
            test_data = data.iloc[test]
            X_train, target_train=train_data[train_data.columns[train_data.columns!='y']], train_data['y']    
            X_test, target_test=test_data[test_data.columns[test_data.columns!='y']], test_data['y']
            model=LogisticRegression(C=n
                                  ,random_state=my_random_state)
            model.fit(X_train,target_train)
            target_predicted=model.predict(X_test)
            my_roc_auc_score=roc_auc_score(target_test, target_predicted)
            my_str='Time elapsed: '+ str(datetime.datetime.now() - start_time)+str(', n=')+str(n)+str(', roc_auc=')+str(my_roc_auc_score)+str(', i=')+str(i)
            i += 1
            auc_all += my_roc_auc_score
            print my_str

        my_str_result= 'Time elapsed: '+ str(datetime.datetime.now() - start_time0)+str(', n=')+str(n)+str(', roc_auc_avg=')+str(auc_all*1.0/max(i,1))
        print my_str_result

Time elapsed: 0:00:01.413394, n=1e-05, roc_auc=0.499770172275, i=0
Time elapsed: 0:00:01.505085, n=1e-05, roc_auc=0.500054106602, i=1
Time elapsed: 0:00:00.631471, n=1e-05, roc_auc=0.500320591476, i=2
Time elapsed: 0:00:00.631940, n=1e-05, roc_auc=0.500424330738, i=3
Time elapsed: 0:00:00.632734, n=1e-05, roc_auc=0.500103335875, i=4
Time elapsed: 0:00:04.825893, n=1e-05, roc_auc_avg=0.500134507393
Time elapsed: 0:00:00.917868, n=0.0001, roc_auc=0.500097855958, i=0
Time elapsed: 0:00:00.917305, n=0.0001, roc_auc=0.499824250276, i=1
Time elapsed: 0:00:00.918626, n=0.0001, roc_auc=0.500187961894, i=2
Time elapsed: 0:00:00.913119, n=0.0001, roc_auc=0.500134611714, i=3
Time elapsed: 0:00:00.917022, n=0.0001, roc_auc=0.499954466806, i=4
Time elapsed: 0:00:04.594348, n=0.0001, roc_auc_avg=0.50003982933
Time elapsed: 0:00:01.530019, n=0.001, roc_auc=0.500149991361, i=0
Time elapsed: 0:00:01.455786, n=0.001, roc_auc=0.499648500552, i=1
Time elapsed: 0:00:01.577238, n=0.001, roc_auc=0.5002472457

2.2

In [90]:
len(data.columns)


92

In [122]:
cols_2_del=['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero']
data2=data.drop(cols_2_del, axis=1)

C_vals=[0.01 , 0.05, 0.1, 0.5, 1, 5, 10, 15]

for c in C_vals:
        i=0
        n=c
        #my_max_depth=3
        my_n_splits=5
        my_shuffle=True
        my_random_state=17
        kf = KFold(n_splits=my_n_splits
                  ,shuffle=my_shuffle
                  ,random_state=my_random_state)
        auc_all=0
        start_time0 = datetime.datetime.now()
        for train, test in kf.split(data2):
            start_time = datetime.datetime.now()
            train_data = data2.iloc[train]
            test_data = data2.iloc[test]
            X_train, target_train=train_data[train_data.columns[train_data.columns!='y']], train_data['y']    
            X_test, target_test=test_data[test_data.columns[test_data.columns!='y']], test_data['y']
            model=LogisticRegression(C=n
                                  ,random_state=my_random_state)
            model.fit(X_train,target_train)
            target_predicted=model.predict(X_test)
            my_roc_auc_score=roc_auc_score(target_test, target_predicted)
            my_str='Time elapsed: '+ str(datetime.datetime.now() - start_time)+str(', n=')+str(n)+str(', roc_auc=')+str(my_roc_auc_score)+str(', i=')+str(i)
            i += 1
            auc_all += my_roc_auc_score
            print my_str

        my_str_result= 'Time elapsed: '+ str(datetime.datetime.now() - start_time0)+str(', n=')+str(n)+str(', roc_auc_avg=')+str(auc_all*1.0/max(i,1))
        print my_str_result

Time elapsed: 0:00:00.357561, n=0.01, roc_auc=0.5, i=0
Time elapsed: 0:00:00.323441, n=0.01, roc_auc=0.5, i=1
Time elapsed: 0:00:00.322493, n=0.01, roc_auc=0.5, i=2
Time elapsed: 0:00:00.300514, n=0.01, roc_auc=0.5, i=3
Time elapsed: 0:00:00.322814, n=0.01, roc_auc=0.5, i=4
Time elapsed: 0:00:01.639586, n=0.01, roc_auc_avg=0.5
Time elapsed: 0:00:00.343396, n=0.05, roc_auc=0.5, i=0
Time elapsed: 0:00:00.376402, n=0.05, roc_auc=0.5, i=1
Time elapsed: 0:00:00.415677, n=0.05, roc_auc=0.5, i=2
Time elapsed: 0:00:00.398639, n=0.05, roc_auc=0.5, i=3
Time elapsed: 0:00:00.386866, n=0.05, roc_auc=0.5, i=4
Time elapsed: 0:00:01.932113, n=0.05, roc_auc_avg=0.5
Time elapsed: 0:00:00.374105, n=0.1, roc_auc=0.5, i=0
Time elapsed: 0:00:00.392856, n=0.1, roc_auc=0.5, i=1
Time elapsed: 0:00:00.347520, n=0.1, roc_auc=0.5, i=2
Time elapsed: 0:00:00.363928, n=0.1, roc_auc=0.5, i=3
Time elapsed: 0:00:00.363321, n=0.1, roc_auc=0.5, i=4
Time elapsed: 0:00:01.852130, n=0.1, roc_auc_avg=0.5
Time elapsed: 0:00:

In [133]:
data['r1_hero'][110:112]

match_id
139    20
142    34
Name: r1_hero, dtype: int64

In [131]:
data.ix[0, 'r1_hero']

11

In [127]:
len(np.unique(data['r1_hero'].append(data['d1_hero'])))

108

In [129]:
import numpy as np
#np.unique(data['r1_hero'], return_counts=True)
len(np.unique(data['r1_hero'])), len(np.unique(data['d1_hero']))


N=len(np.unique(data['d1_hero']))
X_pick = np.zeros((data.shape[0], N))

for i, match_id in enumerate(data.index):
    for p in xrange(5):
        print p, i, data.ix[match_id, 'r%d_hero' % (p+1)]-1, match_id, 'r%d_hero' % (p+1), match_id
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
        

0 0 10 0 r1_hero 0
1 0 66 0 r2_hero 0
2 0 28 0 r3_hero 0
3 0 19 0 r4_hero 0
4 0 104 0 r5_hero 0
0 1 41 1 r1_hero 1
1 1 48 1 r2_hero 1
2 1 66 1 r3_hero 1
3 1 36 1 r4_hero 1
4 1 25 1 r5_hero 1
0 2 32 2 r1_hero 2
1 2 97 2 r2_hero 2
2 2 19 2 r3_hero 2
3 2 26 2 r4_hero 2
4 2 3 2 r5_hero 2
0 3 28 3 r1_hero 3
1 3 29 3 r2_hero 3
2 3 74 3 r3_hero 3
3 3 36 3 r4_hero 3
4 3 40 3 r5_hero 3
0 4 12 4 r1_hero 4
1 4 26 4 r2_hero 4
2 4 29 4 r3_hero 4
3 4 71 4 r4_hero 4
4 4 92 4 r5_hero 4
0 5 10 5 r1_hero 5
1 5 19 5 r2_hero 5
2 5 27 5 r3_hero 5
3 5 24 5 r4_hero 5
4 5 64 5 r5_hero 5
0 6 7 8 r1_hero 8
1 6 56 8 r2_hero 8
2 6 6 8 r3_hero 8
3 6 20 8 r4_hero 8
4 6 35 8 r5_hero 8
0 7 34 9 r1_hero 9
1 7 14 9 r2_hero 9
2 7 82 9 r3_hero 9
3 7 28 9 r4_hero 9
4 7 100 9 r5_hero 9
0 8 16 11 r1_hero 11
1 8 90 11 r2_hero 11
2 8 52 11 r3_hero 11
3 8 71 11 r4_hero 11
4 8 29 11 r5_hero 11
0 9 14 12 r1_hero 12
1 9 40 12 r2_hero 12
2 9 73 12 r3_hero 12
3 9 89 12 r4_hero 12
4 9 41 12 r5_hero 12
0 10 21 14 r1_hero 14
1 10 24 1

IndexError: index 111 is out of bounds for axis 1 with size 108